# Generate ADF4351 registers related funcitons
- https://www.analog.com/media/en/technical-documentation/data-sheets/ADF4351.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [14]:
paths = ['C:\\Users\\Wei\\Dropbox\\Coding\\notebooks\\專案\\待處理\\Signal Generators\\bitbucket\\github\\codes\\signal_generators\\adf435x']
paths = [gen_relative_path(p) for p in paths]
print(paths)

[['..', '..', '..', 'codes', 'signal_generators', 'adf435x']]


In [3]:
paths = [['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],          
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from signal_generators.adf435x.registers_map import _get_registers_map

Populating the interactive namespace from numpy and matplotlib


In [5]:
map = _get_registers_map()

In [33]:
import textwrap



def gen_comment(string, width = 90):
    lines = textwrap.wrap( string,   width = width)
    lines = ['\t# {}'.format(l) for l in lines]
    return '\n'.join(lines)

def remove_comment(line):
    try:
        idx = line.index('#') 
        return line[:idx]
    except:
        return line

def remove_comment_from_file(file_name):
    with open(file_name, encoding = 'utf-8') as f:
        lines = f.readlines()

    for line in lines:
        print(remove_comment(line).replace('\n', ''))
        
def re_wrap_comment(comment, width = 90):
    tab_len = comment.index('#') 
    cm = comment.replace(' ' * tab_len + '# ', '').replace('\n', ' ')    
    lines = textwrap.wrap(cm,   width = width - tab_len)
    comments = [(' ' * tab_len + '# {}').format(l) for l in lines]
    return '\n'.join(comments) 

def gen_set_fun_n_property(registers_map, width = 90):

    def check_char(string):
        string = string.replace('-', '_')
        return string
    
    def gen_set_fun(ele):
        fun_str = "def _set_{}(self, value  = True):\n".format(check_char(ele.name.lower()))
        fun_str += gen_comment(ele.description, width)
        fun_str += '\n\n'
        fun_str += "\tself._adf._write_element_by_name('{}', int(bool(value)))\n".format(ele.name)
        return fun_str

    def gen_property(ele):    
        fun_str = "@property\n"
        fun_str += "def {}(self):\n".format(check_char(ele.name.lower())) 
        # fun_str += gen_comment(ele.description, width)
        # fun_str += '\n\n'
        fun_str += "\treturn self._adf.map.elements['{}']['element'].value\n".format(ele.name)
        return fun_str

    i = 0
    
    for reg in map._registers:

        for ele in reg._elements:

            if ele.name != 'Index':
                print (gen_set_fun(ele))
                print (gen_property(ele)) 
                
                i += 1
    print(i)

In [45]:
comment = '''            # Setting the DB18 bit to 1 enables cycle slip reduction. CSR is a method for improving lock
            # times. Note that the signal at the phase frequency detector (PFD) must have a 50% duty
            # cycle for cycle slip reduction to work. The charge pump current setting must also be set
            # to a minimum. For more information, see the Cycle Slip Reduction for Faster Lock Times
            # section.
'''

print(re_wrap_comment(comment, width = 90))

            # Setting the DB18 bit to 1 enables cycle slip reduction. CSR is a method for
            # improving lock times. Note that the signal at the phase frequency detector
            # (PFD) must have a 50% duty cycle for cycle slip reduction to work. The charge
            # pump current setting must also be set to a minimum. For more information, see
            # the Cycle Slip Reduction for Faster Lock Times section.


In [47]:
file_name = os.path.sep.join(['..', '..', '..', 'codes', 'signal_generators', 'adf435x', 'adf4351.py'])

# remove_comment_from_file(file_name)

In [36]:
# gen_set_fun_n_property(map, width = 90)